In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2) 
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import Activation 
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
import tensorflow_datasets as tfds
from tensorflow.keras.models import load_model

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:21:00.0, compute capability: 8.6



## Import Dataset

In [2]:
dataset_name = "cifar10"
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name, split=["train", "test"], with_info=True, as_supervised=True
)
NUM_CLASSES = ds_info.features["label"].num_classes

IMG_SIZE = 224
batch_size = 64
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label


ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

## Import Model

In [3]:
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = inputs 
model = tf.keras.applications.resnet.ResNet50(weights='imagenet',
                                             input_tensor=x,include_top=False)

model.trainable = False
x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2, name="top_dropout")(x)
outputs = layers.Dense(10, activation="softmax", name="pred")(x)
model = Model(inputs = inputs, outputs = outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

## Transfer Learning on Model

In [4]:
model_name = 'resnet50_cifar.h5'
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(1e-2),
              metrics=['accuracy'])

checkpoint = ModelCheckpoint(model_name, save_best_only=True, monitor='val_accuracy')
model.fit(ds_train,
              epochs=20,
              validation_data=ds_test,
              callbacks=[checkpoint],
              shuffle=True,
              verbose=1)

Epoch 1/20
781/781 [==============================] - 46s 54ms/step - loss: 1.1997 - accuracy: 0.7669 - val_loss: 0.7228 - val_accuracy: 0.8129
Epoch 2/20
781/781 [==============================] - 42s 53ms/step - loss: 0.6258 - accuracy: 0.8133 - val_loss: 0.5087 - val_accuracy: 0.8344
Epoch 3/20
781/781 [==============================] - 42s 53ms/step - loss: 0.5560 - accuracy: 0.8183 - val_loss: 0.5503 - val_accuracy: 0.8244
Epoch 4/20
781/781 [==============================] - 42s 53ms/step - loss: 0.5730 - accuracy: 0.8135 - val_loss: 0.5279 - val_accuracy: 0.8259
Epoch 5/20
781/781 [==============================] - 42s 53ms/step - loss: 0.5798 - accuracy: 0.8111 - val_loss: 0.5321 - val_accuracy: 0.8275
Epoch 6/20
781/781 [==============================] - 42s 53ms/step - loss: 0.5932 - accuracy: 0.8076 - val_loss: 0.5507 - val_accuracy: 0.8237
Epoch 7/20
781/781 [==============================] - 42s 53ms/step - loss: 0.5929 - accuracy: 0.8097 - val_loss: 0.5676 - val_accuracy:

In [5]:
def unfreeze_model(model):
    for layer in model.layers[-15:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )


unfreeze_model(model)

checkpoint = ModelCheckpoint(model_name, save_best_only=True, monitor='val_accuracy')
model.fit(ds_train,
              epochs=20,
              validation_data=ds_test,
              callbacks=[checkpoint],
              shuffle=True,
              verbose=1)

Epoch 1/20
781/781 [==============================] - 47s 57ms/step - loss: 0.4417 - accuracy: 0.8515 - val_loss: 0.3950 - val_accuracy: 0.8660
Epoch 2/20
781/781 [==============================] - 44s 57ms/step - loss: 0.2551 - accuracy: 0.9105 - val_loss: 0.3901 - val_accuracy: 0.8685
Epoch 3/20
781/781 [==============================] - 44s 57ms/step - loss: 0.1399 - accuracy: 0.9518 - val_loss: 0.3636 - val_accuracy: 0.8813
Epoch 4/20
781/781 [==============================] - 44s 57ms/step - loss: 0.0731 - accuracy: 0.9764 - val_loss: 0.3746 - val_accuracy: 0.8831
Epoch 5/20
781/781 [==============================] - 44s 57ms/step - loss: 0.0425 - accuracy: 0.9879 - val_loss: 0.3886 - val_accuracy: 0.8841
Epoch 6/20
781/781 [==============================] - 44s 57ms/step - loss: 0.0358 - accuracy: 0.9894 - val_loss: 0.4180 - val_accuracy: 0.8853
Epoch 7/20
781/781 [==============================] - 44s 57ms/step - loss: 0.0280 - accuracy: 0.9919 - val_loss: 0.4278 - val_accuracy:

In [6]:
new_model = load_model(model_name)

In [7]:
new_model.evaluate(ds_test)

156/156 [==============================] - 8s 44ms/step - loss: 0.5020 - accuracy: 0.8972


[0.5020008087158203, 0.8972355723381042]